In [9]:
%load_ext autoreload
%autoreload
import pandas as pd
import pickle
import sys; sys.path.append('..')
import zarr_helpers
import eodc_hub_role
from titiler_xarray.titiler.xarray.reader import xarray_open_dataset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#!pip install loguru rio-tiler==4.1.11

In [32]:
ds_url = 's3://nasa-eodc-data-store/365_262_262/CMIP6_daily_GISS-E2-1-G_tas.zarr'
variable = 'tas'
reference = False
zoom = 0

In [38]:
zarr_helpers.time_tile_generation(
    zoom=zoom,
    source=ds_url,
    variable=variable,
    reference=reference
)

/home/jovyan/tile-benchmarking/profiling/titiler_xarray/titiler/xarray/reader.py:42: UserWarning: Variable(s) referenced in cell_measures not in variables: ['areacella']
  return xarray.open_dataset(src_path, **xr_open_args)


1246.24

In [10]:
with open('tile_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

In [28]:
ds = xarray_open_dataset(src_path=ds_url, reference=reference)

/home/jovyan/tile-benchmarking/profiling/titiler_xarray/titiler/xarray/reader.py:42: UserWarning: Variable(s) referenced in cell_measures not in variables: ['areacella']
  return xarray.open_dataset(src_path, **xr_open_args)


In [29]:
ds_specs = zarr_helpers.get_dataset_specs(ds_url.split('/')[-1], ds_url, variable, ds)
print(ds_specs)
# number of spatial chunks is going to be number of chunks along the lat dimension by number of chunks along the long dimension
number_spatial_chunks = (ds_specs['shape']['lat']/ds_specs['chunks']['lat']) * (ds_specs['shape']['lon']/ds_specs['chunks']['lon'])
number_spatial_chunks

{'source': 's3://nasa-eodc-data-store/365_262_262/CMIP6_daily_GISS-E2-1-G_tas.zarr', 'collection_name': 'CMIP6_daily_GISS-E2-1-G_tas.zarr', 'variable': 'tas', 'shape': {'time': 730, 'lat': 600, 'lon': 1440}, 'lat_resolution': 0.25, 'lon_resolution': 0.25, 'chunk_size_mb': 95.57746887207031, 'chunks': {'time': 365, 'lat': 262, 'lon': 262}, 'dtype': dtype('float32'), 'number_coord_chunks': 9, 'compression': Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)}


12.586679097954665

In [35]:
zooms = range(12)
values_to_predict = {
    'const': [1],
    'number_of_chunks': [number_spatial_chunks],
    'chunk_size_mb': [ds_specs['chunk_size_mb']]
}
zooms_to_predict = {f'zoom {zoom}': [0] for zoom in zooms}
zooms_to_predict[f'zoom {zoom}'] = [1]
values_to_predict.update(zooms_to_predict)
print(values_to_predict)
loaded_model.predict(pd.DataFrame(values_to_predict))

{'const': [1], 'number_of_chunks': [12.586679097954665], 'chunk_size_mb': [95.57746887207031], 'zoom 0': [1], 'zoom 1': [0], 'zoom 2': [0], 'zoom 3': [0], 'zoom 4': [0], 'zoom 5': [0], 'zoom 6': [0], 'zoom 7': [0], 'zoom 8': [0], 'zoom 9': [0], 'zoom 10': [0], 'zoom 11': [0]}


0    1575.760786
dtype: float64